# Cleaning Data

In [207]:
import time
#from selenium import webdriver
from bs4 import BeautifulSoup
#from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import re
import os
from lxml import etree
import lxml.html, lxml.html.clean
from urllib.parse import urlparse
from urllib.parse import parse_qs
import pandas as pd
from itertools import product
import time
import regex
import numpy as np

#driver = webdriver.Chrome(ChromeDriverManager().install())
verbose_flag = True
# htmlSource = driver.page_source

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

### Load data


In [208]:
path_data_scientist = r"C:\Users\RSYeo\Projects\data\data_scientist_2022_03_03.ftr"
path_data_engineer =r"C:\Users\RSYeo\Projects\data\data_engineer_2022_03_03.ftr"
path_data_analyst = r"C:\Users\RSYeo\Projects\data\data_analyst_2022_03_03.ftr"


files = [path_data_scientist, path_data_engineer, path_data_analyst]

In [209]:
df_data_careers = None

start = True
for file_path in files:
    df_read = pd.read_feather(file_path)

    if(start):
        df_data_careers = df_read.copy()
        start = False

    else:
        df_data_careers = pd.concat([df_data_careers, df_read])

df_data_careers.reset_index(drop=True, inplace=True)
        


### Start cleaning data

#### Format ID

In [210]:
# Format job id to be numerical
df_data_careers['job_id'] = df_data_careers.job_id.astype('str').str.extract("(\d+)")

In [211]:
df_data_careers.job_id.nunique()

7851

In [212]:
df_data_careers['job_title'] = df_data_careers['job_title'].str.lower()

#### Drop non-unique jobs

In [213]:
df_data_careers.drop_duplicates(keep='first', subset=['job_id'], inplace=True)

#### Format salary range

In [214]:
#replace wrong values with nan
df_data_careers.loc[df_data_careers['salary_range'].str.contains('Posted'), 'salary_range'] = np.nan

In [215]:
#clean MYR
df_malaysian_salary = df_data_careers.loc[~df_data_careers['salary_range'].isnull() & df_data_careers['salary_range'].str.contains('MYR'), :]


In [216]:
df_malaysian_salary.head()

,job_id,job_title,salary_range,job_description,job_level,experience,job_type,qualifications,job_specialization,company_name,company_registration,company_size,company_industry,job_link,search_term,state
2,4832121,sql database administrator (数据库管理员),"MYR 15,000 - MYR 25,000","Job DescriptionJob Requirements: Must speak, r...",Senior Executive,3 years,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Computer/Information Technology, IT-Network/Sy...",Morivy Data and Technology Inc.,None,1 - 50 Employees,Computer / Information Technology (Software),/en/job/sql-database-administrator-%E6%95%B0%E...,data scientist,negeri-sembilan
3,4839272,software engineer,"MYR 3,000 - MYR 4,200",Job Description QualificationDegree or higher ...,Junior Executive,2 years,Full-Time,Not Specified,"Computer/Information Technology, IT-Software",TAIACE ENERGY SDN BHD,1006041-X,1 - 50 Employees,Environment/Health/Safety,/en/job/software-engineer-4839272?jobId=jobstr...,data scientist,negeri-sembilan
4,4852008,manufacturing engineer- fresh graduates (1 yea...,"MYR 3,000 - MYR 3,500",Job HighlightsCareer AdvancementDaily Meal All...,Entry Level,None,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Engineering, Electrical, Mechanical",Plexus Manufacturing Sdn Bhd,399136-M,More than 5000 Employees,Electrical & Electronics,/en/job/manufacturing-engineer-fresh-graduates...,data scientist,negeri-sembilan
7,4819995,system support executive,"MYR 1,500 - MYR 2,000",Job DescriptionSystem Support ExecutiveDeliver...,Junior Executive,2 years,Full-Time,Not Specified,"Computer/Information Technology, IT-Software",Infomax Supplies,001423432-A,1 - 50 Employees,Computer/Information Technology (Hardware),/en/job/system-support-executive-4819995?jobId...,data scientist,negeri-sembilan
8,4831057,junior chemist product development laboratory,"MYR 2,500 - MYR 3,300",Job Highlights5-day weekProgressive working en...,Junior Executive,2 years,Full-Time,"Professional Certificate, Diploma, Advanced/Hi...","Sciences, Chemistry",MAPEI MALAYSIA SDN BHD,231780-K,51 - 200 Employees,Manufacturing / Production,/en/job/junior-chemist-product-development-lab...,data scientist,negeri-sembilan


In [217]:
extracted_salary = df_malaysian_salary['salary_range'].str.findall("(\d+,\d+)")

In [218]:
df_malaysian_salary.loc[:,'start_salary_range'] = extracted_salary.str[0]
df_malaysian_salary.loc[:,'end_salary_range'] = extracted_salary.str[1]

C:\Users\RSYeo\AppData\Local\Temp\ipykernel_17104\4207735271.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_malaysian_salary.loc[:,'start_salary_range'] = extracted_salary.str[0]
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_17104\4207735271.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_malaysian_salary.loc[:,'end_salary_range'] = extracted_salary.str[1]


In [219]:
df_malaysian_salary['start_salary_range'] = df_malaysian_salary['start_salary_range'].str.replace(',','')
df_malaysian_salary.loc[:,'end_salary_range'] = df_malaysian_salary.loc[:,'end_salary_range'].str.replace(',','')

C:\Users\RSYeo\AppData\Local\Temp\ipykernel_17104\2981404410.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_malaysian_salary['start_salary_range'] = df_malaysian_salary['start_salary_range'].str.replace(',','')
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_17104\2981404410.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_malaysian_salary.loc[:,'end_salary_range'] = df_malaysian_salary.loc[:,'end_salary_range'].str.replace(',','')


In [220]:
df_malaysian_salary['start_salary_range'] = df_malaysian_salary['start_salary_range'].astype(float)
df_malaysian_salary['end_salary_range'] = df_malaysian_salary['end_salary_range'].astype(float)

C:\Users\RSYeo\AppData\Local\Temp\ipykernel_17104\445547010.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_malaysian_salary['start_salary_range'] = df_malaysian_salary['start_salary_range'].astype(float)
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_17104\445547010.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_malaysian_salary['end_salary_range'] = df_malaysian_salary['end_salary_range'].astype(float)


In [221]:
df_malaysian_salary.shape

(1565, 18)

In [222]:
currencies = df_data_careers['salary_range'].str[:3]

In [223]:
currencies.value_counts()

MYR    1565
SGD       3
HKD       1
Name: salary_range, dtype: int64

In [224]:
#clean SGD
df_singaporean_salary = df_data_careers.loc[~df_data_careers['salary_range'].isnull() & df_data_careers['salary_range'].str.contains('SGD'), :]
extracted_salary_SGD = df_singaporean_salary['salary_range'].str.findall("(\d+,\d+)")
df_singaporean_salary.loc[:,'start_salary_range'] = extracted_salary_SGD.str[0]
df_singaporean_salary.loc[:,'end_salary_range'] = extracted_salary_SGD.str[1]
df_singaporean_salary['start_salary_range'] = df_singaporean_salary['start_salary_range'].str.replace(',','')
df_singaporean_salary.loc[:,'end_salary_range'] = df_singaporean_salary.loc[:,'end_salary_range'].str.replace(',','')

#conversion to MYR
MYR = 3.08
df_singaporean_salary['start_salary_range'] = df_singaporean_salary['start_salary_range'].astype(float)*3.08
df_singaporean_salary['end_salary_range'] = df_singaporean_salary['end_salary_range'].astype(float)*3.08

df_singaporean_salary

C:\Users\RSYeo\AppData\Local\Temp\ipykernel_17104\2162059839.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_singaporean_salary.loc[:,'start_salary_range'] = extracted_salary_SGD.str[0]
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_17104\2162059839.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_singaporean_salary.loc[:,'end_salary_range'] = extracted_salary_SGD.str[1]
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_17104\2162059839.py:6: SettingWithCopyWarning: 
A value is trying to be set on a cop

,job_id,job_title,salary_range,job_description,job_level,experience,job_type,qualifications,job_specialization,company_name,company_registration,company_size,company_industry,job_link,search_term,state,start_salary_range,end_salary_range
0,4839088,big data lead (work in singapore) - 44565,"SGD 7,000 - SGD 10,500",Job HighlightsExciting future with breakthroug...,Manager,7 years,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Computer/Information Technology, IT-Software",Keysight Technologies Malaysia Sdn. Bhd.,463532-M,2001 - 5000 Employees,Electrical & Electronics,/en/job/big-data-lead-work-in-singapore-44565-...,data scientist,negeri-sembilan,21560.0,32340.0
6595,9299289,"senior commodity specialist (based in penang, ...","SGD 2,000 - SGD 3,000",Job DescriptionResponsibilities:Manages the ou...,Junior Executive,None,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Manufacturing, Purchasing/Material Mgmt",ASM Front-End Manufacturing Singapore Pte Ltd,200310868G,201 - 500 Employees,Semiconductor/Wafer Fabrication,/en/job/senior-commodity-specialist-based-in-p...,data analyst,penang,6160.0,9240.0
6713,9299313,"senior buyer (penang, malaysia)","SGD 2,000 - SGD 3,000",Job DescriptionResponsibilities:Responsible fo...,Junior Executive,None,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Manufacturing, Purchasing/Material Mgmt",ASM Front-End Manufacturing Singapore Pte Ltd,200310868G,201 - 500 Employees,Semiconductor/Wafer Fabrication,/en/job/senior-buyer-penang-malaysia-9299313/o...,data analyst,penang,6160.0,9240.0


In [225]:
#clean HKD
df_hongkong_salary = df_data_careers.loc[~df_data_careers['salary_range'].isnull() & df_data_careers['salary_range'].str.contains('HKD'), :]
extracted_salary_HKD = df_hongkong_salary['salary_range'].str.findall("(\d+,\d+)")
df_hongkong_salary.loc[:,'start_salary_range'] = extracted_salary_HKD.str[0]
df_hongkong_salary.loc[:,'end_salary_range'] = extracted_salary_HKD.str[1]
df_hongkong_salary['start_salary_range'] = df_hongkong_salary['start_salary_range'].str.replace(',','')
df_hongkong_salary.loc[:,'end_salary_range'] = df_hongkong_salary.loc[:,'end_salary_range'].str.replace(',','')

#conversion to MYR
MYR = 0.54
df_hongkong_salary['start_salary_range'] = df_hongkong_salary['start_salary_range'].astype(float)*MYR
df_hongkong_salary['end_salary_range'] = df_hongkong_salary['end_salary_range'].astype(float)*MYR

df_hongkong_salary

C:\Users\RSYeo\AppData\Local\Temp\ipykernel_17104\3422634028.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hongkong_salary.loc[:,'start_salary_range'] = extracted_salary_HKD.str[0]
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_17104\3422634028.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hongkong_salary.loc[:,'end_salary_range'] = extracted_salary_HKD.str[1]
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_17104\3422634028.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

,job_id,job_title,salary_range,job_description,job_level,experience,job_type,qualifications,job_specialization,company_name,company_registration,company_size,company_industry,job_link,search_term,state,start_salary_range,end_salary_range
3908,4845280,system analyst / senior analyst programmer (ba...,"HKD 25,000 - HKD 35,000",Job DescriptionAbout this roleRegional support...,Senior Executive,5 years,Full-Time,Not Specified,"Computer/Information Technology, IT-Software",Nexperia Malaysia Sdn.Bhd.,244835-W,2001 - 5000 Employees,Semiconductor/Wafer Fabrication,/en/job/system-analyst-senior-analyst-programm...,data analyst,selangor,13500.0,18900.0


In [226]:
df_data_careers['start_salary_range'] = np.nan
df_data_careers['end_salary_range'] = np.nan

In [230]:
def update_data(main_df:pd.DataFrame, new_df:pd.DataFrame, column_names:list):
    for column_name in column_names:
        main_df.loc[new_df.index, column_name] = new_df[column_name]
    
    return main_df

In [231]:
new_data_dfs = [df_malaysian_salary, df_singaporean_salary, df_hongkong_salary]

for df in new_data_dfs:
    df_data_careers = update_data(df_data_careers, df, ['start_salary_range', 'end_salary_range'])


In [235]:
df_data_careers.drop('salary_range', axis=1, inplace=True)